In [1]:
from confluent_kafka import Producer

In [28]:
class ExampleProducer:
    broker = "localhost:9092"
    producer = None

    def __init__(self, topic):
        self.producer = Producer({
            'bootstrap.servers': self.broker,
            'socket.timeout.ms': 100,
            'api.version.request': 'false',
            'broker.version.fallback': '0.9.0',
        }
        )
        self.topic = topic

    def delivery_report(self, err, msg):
        """ Called once for each message produced to indicate delivery result.
            Triggered by poll() or flush(). """
        if err is not None:
            print('Message delivery failed: {}'.format(err))
        else:
            print('Message delivered to {} [{}]'.format(
                msg.topic(), msg.partition()))

    def send_msg_async(self, msg):
        print("Send message asynchronously")
        self.producer.produce(
            self.topic,
            msg,
            callback=lambda err, original_msg=msg: self.delivery_report(err, original_msg
                                                                        ),
        )
        self.producer.flush()

    def send_msg_sync(self, msg):
        print("Send message synchronously")
        self.producer.produce(
            self.topic,
            msg,
            callback=lambda err, original_msg=msg: self.delivery_report(
                err, original_msg
            ),
        )
        self.producer.flush()



In [7]:
import pandas as pd
from io import BytesIO

In [29]:
#SENDING DATA TO KAFKA TOPIC
import json
from time import sleep, time


example_producer = ExampleProducer("test-topic")
# df = pd.read_parquet("../data/Airports.parquet")
# message = df.to_parquet()
message = ["testing"+i for i in [str(k) for k in range(22)]]
for mes in message :
    example_producer.send_msg_async(mes)
    sleep(1)

Send message asynchronously
Message delivered to test-topic [0]
Send message asynchronously
Message delivered to test-topic [0]


KeyboardInterrupt: 